In [1]:
from elasticsearch import Elasticsearch

In [2]:
ES_URL = "http://elasticsearch:9200"
INDEX = "data_playground_products"

In [3]:
es = Elasticsearch(hosts=[ES_URL])

In [4]:
response = es.count(
    index=INDEX,
    body={
        "query": {"match_all": {}},
    },
)
print(response["count"])

30000


In [5]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match_all": {}},
        "size": 3,
    },
)
print(
    *(r["_source"]["pid"] for r in response["hits"]["hits"]),
    sep="\n",
)

TSHFPS9EHSSHUWVU
SHTFPS228JPGPGHH
TSHFESTUMGXZCJGZ


In [6]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "casual"}},
        "size": 3,
    },
)
print(
    *(r["_source"]["title"] for r in response["hits"]["hits"]),
    sep="\n",
)

Men Brown Casual Sandal
Men Tan Casual Sandal
Men Beige Casual Sandal


In [7]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"description": "goku"}},
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [8]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "multi_match": {
                "query": "ball",
                "fields": ["title", "description"],
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["title"], r["_source"]["description"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Base Ball Cap Cap', 'Tee Buddy Caps are Comfortable fit for both men and women, Suitable for wearing at all occasions, Fabric is pre-washed before stitching. No color Fade for a long time, Total Length - 25 cm, Hood Width - 15 cm, Height - 13 cm')
('JAGUAR Football Shoes For Men\xa0\xa0(Black, Orange)', 'VECTOR-X \xa0JAGUAR Soccer Cleat provides exceptional ball touch and a comfortable,secure fit that promotes acceleration and quick directional changes on multiple kinds of surfaces , Dynamic Fit collar for secure comfort. Synthetic and textile upper for durable play.')
('Printed Men Round Neck Black T-Shirt', 'Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!')


In [9]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must": [
                    {"range": {"average_rating": {"gt": 3}}},
                    {"match": {"category": "Clothing"}},
                ],
                "must_not": {
                    "range": {"discount": {"lte": 75}},
                },
                "should": {
                    "match": {"brand": "Dex"},
                },
            },
        },
        "size": 3,
    },
)
print(
    *(
        (
            r["_source"]["brand"],
            r["_source"]["average_rating"],
            r["_source"]["category"],
            r["_source"]["discount"],
        )
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Amp', 3.4, 'Clothing and Accessories', 76.0)
('Amp', 3.4, 'Clothing and Accessories', 76.0)
('Amp', 3.4, 'Clothing and Accessories', 80.0)


In [10]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "term": {"description": "goku"},
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [11]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "term": {"description": "gok"},
        },
        "size": 3,
    },
)
out = [r["_source"]["description"] for r in response["hits"]["hits"]] or ["None"]
print(
    *out,
    sep="\n",
)

None


In [12]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "range": {"selling_price": {"lt": 100}},
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["selling_price"] for r in response["hits"]["hits"]),
    sep="\n",
)

99.0
99.0


In [13]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "filter": [
                    {"term": {"out_of_stock": False}},
                ],
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["out_of_stock"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('TSHFPS9EHSSHUWVU', False)
('SHTFPS228JPGPGHH', False)
('TSHFESTUMGXZCJGZ', False)


In [14]:
response = es.search(
    index=INDEX,
    body={
        "aggs": {
            "categories": {
                "terms": {"field": "category.keyword"},
            },
        },
    },
)
print(response["aggregations"])

{'categories': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'Clothing and Accessories', 'doc_count': 28971}, {'key': 'Footwear', 'doc_count': 987}, {'key': 'Bags, Wallets & Belts', 'doc_count': 41}, {'key': 'Toys', 'doc_count': 1}]}}


In [15]:
response = es.search(
    index=INDEX,
    body={
        "sort": {"average_rating": {"order": "desc"}},
        "query": {"match_all": {}},
        "size": 3,
    },
)
print(
    *(r["_source"]["average_rating"] for r in response["hits"]["hits"]),
    sep="\n",
)

5.0
5.0
5.0


In [16]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match_all": {}},
        "from": 0,
        "size": 5,
    },
)
print(
    *(r["_source"]["pid"] for r in response["hits"]["hits"]),
    sep="\n",
)

TSHFPS9EHSSHUWVU
SHTFPS228JPGPGHH
TSHFESTUMGXZCJGZ
TSHFQ4FWQ42YYX9F
TSHFSYF7N8YZTGN4


In [17]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match_all": {}},
        "from": 2,
        "size": 3,
    },
)
print(
    *(r["_source"]["pid"] for r in response["hits"]["hits"]),
    sep="\n",
)

TSHFESTUMGXZCJGZ
TSHFQ4FWQ42YYX9F
TSHFSYF7N8YZTGN4


In [18]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<em>Shirt</em>']}


In [19]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "tags_schema": "styled",
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<em class="hlt1">Shirt</em>']}


In [20]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "pre_tags": ["<tag1>"],
            "post_tags": ["</tag1>"],
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<tag1>Shirt</tag1>']}


In [21]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "type": "plain",
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<em>Shirt</em>']}


In [22]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "filter": [
                            {"term": {"out_of_stock": False}},
                        ],
                    },
                },
                "script": {
                    "source": "doc['average_rating'].size() == 0 || "
                    "doc['selling_price'].size() == 0 ? "
                    "0 : "
                    "doc['selling_price'].value / "
                    "(doc['average_rating'].value)",
                },
            },
        },
        "size": 3,
    },
)
print(
    *(
        (
            r["_source"]["out_of_stock"],
            r["_source"]["average_rating"],
            r["_source"]["selling_price"],
            r["_source"]["selling_price"] / r["_source"]["average_rating"],
        )
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

(False, 2.6, 7998.0, 3076.153846153846)
(False, 2.6, 6998.0, 2691.5384615384614)
(False, 2.6, 5998.0, 2306.9230769230767)


In [23]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "more_like_this": {
                "fields": ["title", "description"],
                "like": "dark pants",
                "min_term_freq": 1,
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["title"], r["_source"]["description"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Solid Men Dark Blue Track Pants', "Men's Navy Stripped Track Pant")
('Solid Men Dark Blue Track Pants', 'A2Z Quality Group - Joggers / Track pants have been developed for those who like a form fitting pant with a full stretch! it is higher elastic fabric, they retain the same look as our normal training pants yet possess the ability to stretch and rebound much further. If you squat or train a lot in pants, these ones are for you! , and Training with High Quality Fabric Advanced Stretch fit, Elastic in waist band to give a comfortable grip. Please check the measurement before you buy the item.')
('Printed Men Dark Green Track Pants', '')


In [24]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "more_like_this": {
                "fields": ["title", "description"],
                "like": "slim shirt",
                "min_term_freq": 1,
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["title"], r["_source"]["description"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Men Slim Fit Printed Slim Collar Casual Shirt', '')
('Men Slim Fit Printed Slim Collar Casual Shirt', '')
('Men Slim Fit Solid Slim Collar Casual Shirt', '')


In [25]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "exists": {"field": "product_details.Country of Origin"},
        },
        "aggs": {
            "unique_countries": {
                "terms": {"field": "product_details.Country of Origin.keyword"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["key"] for r in response["aggregations"]["unique_countries"]["buckets"]),
    sep="\n",
)

India
China
Namibia
Bangladesh
Sri Lanka
Hong Kong
Vietnam
Cambodia
Turkey
Madagascar


In [26]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "prefix": {
                "seller": {"value": "ac"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["seller"] for r in response["hits"]["hits"]),
    sep="\n",
)

STYLE ACCORD
STYLE ACCORD
STYLE ACCORD


In [27]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "exists": {"field": "discount"},
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["discount"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('TSHFPS9EHSSHUWVU', 60.0)
('SHTFPS228JPGPGHH', 60.0)
('TSHFESTUMGXZCJGZ', 60.0)


In [28]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must_not": {
                    "exists": {"field": "discount"},
                },
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["discount"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

In [29]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "crawled_at": {
                                "gte": "2021-02-10",
                                "lt": "2021-02-11",
                            },
                        },
                    },
                ],
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["crawled_at"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('TSHFPS9EHSSHUWVU', '2021-02-10T22:14:00+00:00')
('SHTFPS228JPGPGHH', '2021-02-10T22:14:00+00:00')
('TSHFESTUMGXZCJGZ', '2021-02-10T22:14:01+00:00')


In [30]:
response = es.count(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "crawled_at": {
                                "gte": "2021-02-10",
                                "lt": "2021-02-11",
                            },
                        },
                    },
                ],
            },
        },
        # "size": 3,
    },
)
print(response["count"])

20913


In [31]:
response = es.count(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "crawled_at": {
                                "gte": "2021-02-11",
                                "lt": "2021-02-12",
                            },
                        },
                    },
                ],
            },
        },
    },
)
print(response["count"])

9087


In [32]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "functions": [
                    {
                        "linear": {
                            "crawled_at": {
                                "origin": "2021-02-10T22:15:26+00:00",
                                "scale": "1s",
                                "offset": "0s",
                                "decay": 0.5,
                            },
                        },
                    },
                ],
                "score_mode": "multiply",
            },
        },
        "size": 25,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["crawled_at"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('SHTFNNZ56GQGM4YF', '2021-02-10T22:15:26+00:00')
('TSHFPDRGJTZ8AFGG', '2021-02-10T22:15:26+00:00')
('TROFZ9VN2TKMQWGR', '2021-02-10T22:15:25+00:00')
('TSHFM75S4XFHGFSK', '2021-02-10T22:15:27+00:00')
('TSHFPS9EHSSHUWVU', '2021-02-10T22:14:00+00:00')
('SHTFPS228JPGPGHH', '2021-02-10T22:14:00+00:00')
('TSHFESTUMGXZCJGZ', '2021-02-10T22:14:01+00:00')
('TSHFQ4FWQ42YYX9F', '2021-02-10T22:14:02+00:00')
('TSHFSYF7N8YZTGN4', '2021-02-10T22:14:02+00:00')
('SHTFZZFERN6EPZNM', '2021-02-10T22:14:03+00:00')
('TSHFSYF7YZMRUHCP', '2021-02-10T22:14:03+00:00')
('TROFWBXXYFGG7REG', '2021-02-10T22:14:04+00:00')
('TSHFESTU8MW59FJG', '2021-02-10T22:14:04+00:00')
('TSHFEKSRABFHGVFV', '2021-02-10T22:14:05+00:00')
('JCKFP7F7ZHKYAJGZ', '2021-02-10T22:14:06+00:00')
('TSHFNZGGGNCEZU3Z', '2021-02-10T22:14:06+00:00')
('TSHFZRESUHK4HGWS', '2021-02-10T22:14:07+00:00')
('TROFNNPGWCAZDYZT', '2021-02-10T22:14:07+00:00')
('SHTFPS22UYWBFARG', '2021-02-10T22:14:08+00:00')
('TSHFNMKWRSY6VJHF', '2021-02-10T22:14:09+00:00')


In [33]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "functions": [
                    {
                        "exp": {
                            "crawled_at": {
                                "origin": "2021-02-10T22:15:26+00:00",
                                "scale": "1s",
                                "offset": "0s",
                                "decay": 0.5,
                            },
                        },
                    },
                ],
                "score_mode": "multiply",
            },
        },
        "size": 25,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["crawled_at"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('SHTFNNZ56GQGM4YF', '2021-02-10T22:15:26+00:00')
('TSHFPDRGJTZ8AFGG', '2021-02-10T22:15:26+00:00')
('TROFZ9VN2TKMQWGR', '2021-02-10T22:15:25+00:00')
('TSHFM75S4XFHGFSK', '2021-02-10T22:15:27+00:00')
('SHTFEKRVN8GX94ZQ', '2021-02-10T22:15:28+00:00')
('JEAFZRDT6EZSXZZZ', '2021-02-10T22:15:28+00:00')
('TSHFNMKWFTFGSNGD', '2021-02-10T22:15:29+00:00')
('SHTFZZFERGEJXGY4', '2021-02-10T22:15:29+00:00')
('SHTFZRCGHQ8KXUGX', '2021-02-10T22:15:30+00:00')
('TROFM47GBNHWWPZA', '2021-02-10T22:15:30+00:00')
('TSHFPY9ZZ6JDZBNK', '2021-02-10T22:15:31+00:00')
('TSHFPY9ZXWC7AG8S', '2021-02-10T22:15:31+00:00')
('JEAFESNDY7XNYQF3', '2021-02-10T22:15:32+00:00')
('TROFZ9VN5WX44TMK', '2021-02-10T22:15:32+00:00')
('TSHFM75S5RPQ5ZZP', '2021-02-10T22:15:33+00:00')
('TSHFM75S8YXH3MCV', '2021-02-10T22:15:34+00:00')
('JEAFESND4PHGYF3F', '2021-02-10T22:15:34+00:00')
('TSHFWYA4ZB4V9ZSP', '2021-02-10T22:15:36+00:00')
('TSHFM75SDRFVNUTC', '2021-02-10T22:15:37+00:00')
('TSHFZPFJQFFYQ6ZA', '2021-02-10T22:15:37+00:00')


In [34]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "functions": [
                    {
                        "gauss": {
                            "crawled_at": {
                                "origin": "2021-02-10T22:15:26+00:00",
                                "scale": "1s",
                                "offset": "0s",
                                "decay": 0.5,
                            },
                        },
                    },
                ],
                "score_mode": "multiply",
            },
        },
        "size": 25,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["crawled_at"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('SHTFNNZ56GQGM4YF', '2021-02-10T22:15:26+00:00')
('TSHFPDRGJTZ8AFGG', '2021-02-10T22:15:26+00:00')
('TROFZ9VN2TKMQWGR', '2021-02-10T22:15:25+00:00')
('TSHFM75S4XFHGFSK', '2021-02-10T22:15:27+00:00')
('SHTFEKRVN8GX94ZQ', '2021-02-10T22:15:28+00:00')
('JEAFZRDT6EZSXZZZ', '2021-02-10T22:15:28+00:00')
('TSHFNMKWFTFGSNGD', '2021-02-10T22:15:29+00:00')
('SHTFZZFERGEJXGY4', '2021-02-10T22:15:29+00:00')
('SHTFZRCGHQ8KXUGX', '2021-02-10T22:15:30+00:00')
('TROFM47GBNHWWPZA', '2021-02-10T22:15:30+00:00')
('TSHFPY9ZZ6JDZBNK', '2021-02-10T22:15:31+00:00')
('TSHFPY9ZXWC7AG8S', '2021-02-10T22:15:31+00:00')
('JEAFESNDY7XNYQF3', '2021-02-10T22:15:32+00:00')
('TROFZ9VN5WX44TMK', '2021-02-10T22:15:32+00:00')
('TSHFM75S5RPQ5ZZP', '2021-02-10T22:15:33+00:00')
('TSHFM75S8YXH3MCV', '2021-02-10T22:15:34+00:00')
('JEAFESND4PHGYF3F', '2021-02-10T22:15:34+00:00')
('TSHFWYA4ZB4V9ZSP', '2021-02-10T22:15:36+00:00')
('TSHFM75SDRFVNUTC', '2021-02-10T22:15:37+00:00')
('TSHFZPFJQFFYQ6ZA', '2021-02-10T22:15:37+00:00')


In [35]:
response = es.search(
    index=INDEX,
    body={
        "aggs": {
            "title_hits": {
                "terms": {
                    "field": "title.keyword",
                    "size": 3,
                },
                "aggs": {
                    "top_title_documents": {
                        "top_hits": {
                            "sort": [
                                {"average_rating": {"order": "desc"}},
                            ],
                            "size": 3,
                            "_source": {
                                "includes": ["pid"],
                            },
                        },
                    },
                },
            },
        },
    },
)
print(
    *(
        (
            r["key"],
            r["doc_count"],
            [i["_source"]["pid"] for i in r["top_title_documents"]["hits"]["hits"]],
        )
        for r in response["aggregations"]["title_hits"]["buckets"]
    ),
    sep="\n",
)

('Printed Men Round Neck Black T-Shirt', 856, ['TSHFU5AF8KZC5FYH', 'TSHFTT2SWMZ4GA8D', 'TSHFSPXRZNZEQZAV'])
('Printed Men Round Neck White T-Shirt', 703, ['TSHFKZUQNFGATATN', 'TSHFKYKQQ9ZQF44K', 'TSHEHS7HGEVKDWMM'])
('Slides', 556, ['SFFFWZFUD4KHRYRH', 'SFFFUZ95PYJ86ZGC', 'SFFFUZ952RZYMNZY'])


In [36]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "wildcard": {
                "description": {"value": "*goku*"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [37]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "wildcard": {
                "description": {"value": "*go?u*"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

DARK INDIGOFULL SLEEVESMENS SHIRT
Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [38]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "wildcard": {
                "description": {"value": "go?u"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [39]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must": [{"exists": {"field": "description"}}],
                "must_not": [{"term": {"description.keyword": ""}}],
                "should": {
                    "match": {"description": "goku"},
                },
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!
Give a cool upgrade to your rotation of casuals with this polo shirt from True Blue. The breathable cotton-modal construction allows day-long comfort while a ribbed collar and sleeve hems complete the design. Team it with cuffed chinos and sneakers.
True Blue's striped polo shirt adds a touch of colour and pattern to casual line-ups. It ensures you're cool and comfy with a light cotton feel and short-sleeve design. Wear yours with cuffed shorts on warm days.


In [40]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "constant_score": {
                "filter": {
                    "match": {"description": "goku"},
                },
                "boost": 1.2,
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [41]:
response = es.search(
    index=INDEX,
    body={
        "aggs": {
            "stepped_ranges": {
                "range": {
                    "field": "selling_price",
                    "ranges": [
                        {"from": 0, "to": 1000},
                        {"from": 1000, "to": 2000},
                        {"from": 2000, "to": 3000},
                        {"from": 3000, "to": 4000},
                        {"from": 5000},
                    ],
                },
            },
        },
    },
)
print(
    *(
        (r["key"], r["doc_count"])
        for r in response["aggregations"]["stepped_ranges"]["buckets"]
    ),
    sep="\n",
)

('0.0-1000.0', 25478)
('1000.0-2000.0', 3505)
('2000.0-3000.0', 706)
('3000.0-4000.0', 187)
('5000.0-*', 44)


In [42]:
response = es.search(
    index=INDEX,
    body={
        "aggs": {
            "group_by_category": {
                "terms": {"field": "category.keyword"},
                "aggs": {
                    "group_by_subcategory": {
                        "terms": {"field": "sub_category.keyword"},
                    },
                },
            },
        },
    },
)
print(
    *(
        (
            r["key"],
            r["doc_count"],
            [(s["key"], s["doc_count"]) for s in r["group_by_subcategory"]["buckets"]],
        )
        for r in response["aggregations"]["group_by_category"]["buckets"]
    ),
    sep="\n",
)

('Clothing and Accessories', 28971, [('Topwear', 16575), ('Bottomwear', 3862), ('Winter Wear', 2753), ('Innerwear and Swimwear', 1774), ('Clothing Accessories', 1760), ('Kurtas, Ethnic Sets and Bottoms', 1248), ('Fabrics', 535), ('Blazers, Waistcoats and Suits', 136), ('Sleepwear', 122), ('Tracksuits', 90)])
('Footwear', 987, [("Men's Footwear", 987)])
('Bags, Wallets & Belts', 41, [('Sunshopping Bags, Wallets & Belts', 21), ('SUNSHOPPING Bags, Wallets & Belts', 10), ('Winsome Deal Bags, Wallets & Belts', 4), ('YOFAMA Bags, Wallets & Belts', 4), ('Brand Trunk Bags, Wallets & Belts', 1), ('mentiezi Bags, Wallets & Belts', 1)])
('Toys', 1, [('Party Supplies', 1)])


In [43]:
response = es.search(
    index=INDEX,
    body={
        "aggs": {
            "group_by_brand": {
                "terms": {"field": "brand.keyword"},
                "aggs": {
                    "group_by_category": {
                        "terms": {"field": "category.keyword"},
                    },
                },
            },
        },
    },
)
print(
    *(
        (
            r["key"],
            r["doc_count"],
            [(s["key"], s["doc_count"]) for s in r["group_by_category"]["buckets"]],
        )
        for r in response["aggregations"]["group_by_brand"]["buckets"]
    ),
    sep="\n",
)

('', 2068, [('Clothing and Accessories', 2026), ('Bags, Wallets & Belts', 41), ('Toys', 1)])
('ARBO', 999, [('Clothing and Accessories', 999)])
('Pu', 996, [('Clothing and Accessories', 996)])
('REEB', 996, [('Clothing and Accessories', 996)])
('True Bl', 996, [('Clothing and Accessories', 996)])
('ECKO Unl', 993, [('Clothing and Accessories', 993)])
('Free Authori', 864, [('Clothing and Accessories', 864)])
('Keo', 668, [('Clothing and Accessories', 668)])
('Amp', 621, [('Clothing and Accessories', 621)])
('Black Beat', 560, [('Footwear', 560)])
